In [1]:
# Load data
# Clean data
# Feature Enginnering
# Visualizing
# Modelling
# Predict to test data
# Submit

In [2]:
!ls

00.ipynb  data	sample_submission.csv  sub-0.csv  test.csv  train.csv  wandb


In [3]:
# Imports

In [4]:
import torch,torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wandb
from ray import tune

In [5]:
# Load data

In [6]:
!ls ./data/

tabular-playground-series-jul-2021.zip


In [7]:
# !unzip ./data/tabular-playground-series-jul-2021.zip

In [8]:
test_df = pd.read_csv('./test.csv')
data = pd.read_csv('./train.csv')
sample_submission = pd.read_csv('./sample_submission.csv')

In [9]:
# Clean data

In [10]:
data.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


In [11]:
data.isna().sum()

date_time                 0
deg_C                     0
relative_humidity         0
absolute_humidity         0
sensor_1                  0
sensor_2                  0
sensor_3                  0
sensor_4                  0
sensor_5                  0
target_carbon_monoxide    0
target_benzene            0
target_nitrogen_oxides    0
dtype: int64

In [12]:
data.dtypes

date_time                  object
deg_C                     float64
relative_humidity         float64
absolute_humidity         float64
sensor_1                  float64
sensor_2                  float64
sensor_3                  float64
sensor_4                  float64
sensor_5                  float64
target_carbon_monoxide    float64
target_benzene            float64
target_nitrogen_oxides    float64
dtype: object

In [13]:
len(data)

7111

In [14]:
dates = []
times = []
for date_time in data['date_time']:
    date,time = date_time.split(' ')
    time = time.replace(':','')
    date = date.replace('-','')
    dates.append(date)
    times.append(time)

In [15]:
data = data.drop('date_time',axis=1)

In [16]:
data['date'] = dates

In [17]:
data['time'] = times

In [18]:
data.head()

,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides,date,time
0,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7,20100310,180000
1,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9,20100310,190000
2,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1,20100310,200000
3,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2,20100310,210000
4,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8,20100310,220000


In [19]:
data.dtypes

deg_C                     float64
relative_humidity         float64
absolute_humidity         float64
sensor_1                  float64
sensor_2                  float64
sensor_3                  float64
sensor_4                  float64
sensor_5                  float64
target_carbon_monoxide    float64
target_benzene            float64
target_nitrogen_oxides    float64
date                       object
time                       object
dtype: object

In [20]:
data['date'] = data['date'].astype(int)
data['time'] = data['time'].astype(int)

In [21]:
data.head()

,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides,date,time
0,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7,20100310,180000
1,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9,20100310,190000
2,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1,20100310,200000
3,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2,20100310,210000
4,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8,20100310,220000


In [22]:
data.dtypes

deg_C                     float64
relative_humidity         float64
absolute_humidity         float64
sensor_1                  float64
sensor_2                  float64
sensor_3                  float64
sensor_4                  float64
sensor_5                  float64
target_carbon_monoxide    float64
target_benzene            float64
target_nitrogen_oxides    float64
date                        int64
time                        int64
dtype: object

In [23]:
# Feature Enginnering

In [24]:
cols = data.drop(['target_carbon_monoxide','target_benzene','target_nitrogen_oxides'],axis=1)

In [25]:
cols = list(cols.columns)

In [26]:
def feature_enginnerg(data,col):
    print(len(data))
    max_no = data[col].quantile(0.99) # 0.95
    min_no = data[col].quantile(0.05)
    data = data[data[col] > min_no]
    data = data[data[col] < max_no]
    print(len(data))
    return data

In [27]:
data_just = data

In [28]:
feature_enginnering_or_not = False

In [29]:
if feature_enginnering_or_not:
    for col in cols:
        print('*'*50)
        print(col)
        data = feature_enginnerg(data,col)
        print(col)
        print('*'*50)

In [30]:
# Modelling

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X,y = data.drop(data[['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']],axis=1),data[['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']]

In [33]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125)

In [34]:
batch_size = 32
epochs = 100

In [35]:
device = 'cuda'

In [36]:
from tqdm import tqdm

In [37]:
def test(X_test,y_test,model):
    X_test = torch.tensor(np.array(X_test))
    y_test = torch.tensor(np.array(y_test))
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for idx in range(len(X_test)):
            preds = model(X_test[idx].to(device).float().view(1,10))
            results = np.round(y_test[idx].float().cpu()) == np.round(preds.float()[0].cpu())
            if any(results):
                correct += 1
            total += 1
    model.train()
    return round(correct/total,3)

In [38]:
class Model(nn.Module):
    def __init__(self,starter=64,activation=nn.ReLU()):
        super().__init__()
        self.fc1 = nn.Linear(10,starter)
        self.fc1batchnorm = nn.BatchNorm1d(starter)
        self.fc2 = nn.Linear(starter,starter*2)
        self.fc2batchnorm = nn.BatchNorm1d(starter*2)
        self.fc3 = nn.Linear(starter*2,starter*4)
        self.fc3batchnorm = nn.BatchNorm1d(starter*4)
        self.fc4 = nn.Linear(starter*4,starter*2)
        self.fc4batchnorm = nn.BatchNorm1d(starter*2)
        self.output = nn.Linear(starter*2,3)
        self.activation = activation
        
    def forward(self,X):
        preds = self.activation(self.fc1batchnorm(self.fc1(X)))
        preds = self.activation(self.fc2batchnorm(self.fc2(preds)))
        preds = self.activation(self.fc3batchnorm(self.fc3(preds)))
        preds = self.activation(self.fc4batchnorm(self.fc4(preds)))
        preds = self.output(preds)
        return preds

In [39]:
PROJECT_NAME = 'July-Tabular'

In [40]:
def train(config,tune_using=True,name='test'):
    if tune_using is False:
        wandb.init(project=PROJECT_NAME,name=name)
    epochs = config['epochs']
    batch_size = config['batch_size']
    criterion = config['criterion']()
    model = Model(starter=config['starter'],activation=config['activation']).to(device)
    optimizer = config['optimizer'](model.parameters(),lr=config['lr'])
    for epoch in tqdm(range(epochs)):
        for idx in range(0,len(X_train),batch_size):
            X_batch = torch.tensor(np.array(X_train)[idx:idx+batch_size]).to(device)
            y_batch = torch.tensor(np.array(y_train)[idx:idx+batch_size]).to(device)
            model.to(device)
            preds = model(X_batch.float()).to(device)
            loss = criterion(preds.float(),y_batch.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if tune_using is False:
                wandb.log({'loss':loss.item()})
            if tune_using:
                tune.report(mean_loss=loss.item(),accuracy=test(X_test,y_test,model))
        if tune_using is False:
            wandb.log({'val_accuracy':test(X_test,y_test,model)})
    if tune_using is False:
        return model

In [41]:
# config = {'epochs':100,'batch_size':32,'criterion':nn.MSELoss,'starter':64,'activation':nn.ReLU(),'optimizer':optim.Adam,'lr':0.0001}
# model = train(config,False,'baseline-add-1-more-layer')

In [42]:
# config = {'epochs':100,'batch_size':32,'criterion':nn.MSELoss,'starter':64,'activation':nn.ReLU(),'optimizer':optim.Adam,'lr':0.001}
# model = train(config,False,'baseline')

In [43]:
analysis = tune.run(
    train,
    config={
        "epochs": tune.grid_search([25]),
        "batch_size": tune.grid_search([len(X_train)]),
        "criterion": tune.grid_search([nn.MSELoss,nn.L1Loss]),
        "starter": tune.grid_search([16,32,64,128,256,512]),
        "activation": tune.grid_search([nn.LeakyReLU(),nn.PReLU(),nn.ReLU(),nn.ReLU6(),nn.RReLU(),nn.ELU(),nn.SiLU()]),
        'optimizer':tune.grid_search([optim.Adam,optim.AdamW,optim.Adamax,optim.Adagrad,optim.SGD]),
        'lr':tune.grid_search([1e-3,1e-4,1e-2,1e-1,1e-5])
    },resources_per_trial={"gpu": 1},)
print("Best config: ", analysis.get_best_config(
    metric="mean_loss", mode="min"))
df = analysis.results_df
df.head()

2021-07-07 13:18:20,136	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265
2021-07-07 13:18:21,855	WARNING function_runner.py:545 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,activation,batch_size,criterion,epochs,lr,optimizer,starter
train_b3bfb_00000,PENDING,,ReLU6(),6222,<class 'torch.nn.modules.loss.MSELoss'>,25,0.001,<class 'torch.optim.adagrad.Adagrad'>,64
train_b3bfb_00001,PENDING,,"RReLU(lower=0.125, upper=0.3333333333333333)",6222,<class 'torch.nn.modules.loss.L1Loss'>,25,0.0001,<class 'torch.optim.sgd.SGD'>,512


  0%|          | 0/25 [00:00<?, ?it/s]
Result for train_b3bfb_00000:
  accuracy: 0.11
  date: 2021-07-07_13-18-26
  done: false
  experiment_id: 3931e1a377ab4b57ac65c5a274772c15
  hostname: ranugad
  iterations_since_restore: 1
  mean_loss: 26466.654296875
  neg_mean_loss: -26466.654296875
  node_ip: 192.168.1.9
  pid: 752361
  time_since_restore: 2.8707375526428223
  time_this_iter_s: 2.8707375526428223
  time_total_s: 2.8707375526428223
  timestamp: 1625644106
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: b3bfb_00000
  
 12%|█▏        | 3/25 [00:01<00:11,  1.88it/s]


Trial name,status,loc,activation,batch_size,criterion,epochs,lr,optimizer,starter,loss,iter,total time (s),accuracy,neg_mean_loss
train_b3bfb_00000,RUNNING,192.168.1.9:752361,ReLU6(),6222,<class 'torch.nn.modules.loss.MSELoss'>,25,0.001,<class 'torch.optim.adagrad.Adagrad'>,64,26375.2,3,3.92332,0.11,-26375.2
train_b3bfb_00001,PENDING,,"RReLU(lower=0.125, upper=0.3333333333333333)",6222,<class 'torch.nn.modules.loss.L1Loss'>,25,0.0001,<class 'torch.optim.sgd.SGD'>,512,,,,,


 40%|████      | 10/25 [00:05<00:07,  1.91it/s]
Result for train_b3bfb_00000:
  accuracy: 0.085
  date: 2021-07-07_13-18-31
  done: false
  experiment_id: 3931e1a377ab4b57ac65c5a274772c15
  hostname: ranugad
  iterations_since_restore: 11
  mean_loss: 26208.6640625
  neg_mean_loss: -26208.6640625
  node_ip: 192.168.1.9
  pid: 752361
  time_since_restore: 8.125495433807373
  time_this_iter_s: 0.5219604969024658
  time_total_s: 8.125495433807373
  timestamp: 1625644111
  timesteps_since_restore: 0
  training_iteration: 11
  trial_id: b3bfb_00000
  
 48%|████▊     | 12/25 [00:06<00:06,  1.91it/s]


...

Trial name,status,loc,activation,batch_size,criterion,epochs,lr,optimizer,starter,loss,iter,total time (s),accuracy,neg_mean_loss
train_b3bfb_00001,RUNNING,192.168.1.9:752362,"RReLU(lower=0.125, upper=0.3333333333333333)",6222,<class 'torch.nn.modules.loss.L1Loss'>,25,0.0001,<class 'torch.optim.sgd.SGD'>,512,72.1162,19,13.0784,0.04,-72.1162
train_b3bfb_00000,TERMINATED,,ReLU6(),6222,<class 'torch.nn.modules.loss.MSELoss'>,25,0.001,<class 'torch.optim.adagrad.Adagrad'>,64,26154.8,25,15.4449,0.087,-26154.8


 96%|█████████▌| 24/25 [00:13<00:00,  1.76it/s]
2021-07-07 13:18:56,172	INFO logger.py:699 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': RReLU(lower=0.125, upper=0.3333333333333333), 'optimizer': <class 'torch.optim.sgd.SGD'>, 'criterion': <class 'torch.nn.modules.loss.L1Loss'>}
Result for train_b3bfb_00001:
  accuracy: 0.002
  date: 2021-07-07_13-18-56
  done: true
  experiment_id: 884403f1fd6b4abd81b4c82de10987ec
  experiment_tag: 1_activation=RReLU(lower=0.125, upper=0.3333333333333333),batch_size=6222,criterion=<class
    'torch.nn.modules.loss.L1Loss'>,epochs=25,lr=0.0001,optimizer=<class 'torch.optim.sgd.SGD'>,starter=512
  hostname: ranugad
  iterations_since_restore: 25
  mean_loss: 72.09603881835938
  neg_mean_loss: -72.09603881835938
  node_ip: 192.168.1.9
  pid: 752362
  time_since_restore: 16.479299306869507
  time_this_iter_s: 0.5490083694458008
  time_total_s: 16.479299306869507
  timestamp: 1625644136
  timesteps_since_restore

Trial name,status,loc,activation,batch_size,criterion,epochs,lr,optimizer,starter,loss,iter,total time (s),accuracy,neg_mean_loss
train_b3bfb_00000,TERMINATED,,ReLU6(),6222,<class 'torch.nn.modules.loss.MSELoss'>,25,0.001,<class 'torch.optim.adagrad.Adagrad'>,64,26154.8,25,15.4449,0.087,-26154.8
train_b3bfb_00001,TERMINATED,,"RReLU(lower=0.125, upper=0.3333333333333333)",6222,<class 'torch.nn.modules.loss.L1Loss'>,25,0.0001,<class 'torch.optim.sgd.SGD'>,512,72.096,25,16.4793,0.002,-72.096


2021-07-07 13:18:56,303	INFO tune.py:549 -- Total run time: 34.45 seconds (34.26 seconds for the tuning loop).
Best config:  {'epochs': 25, 'batch_size': 6222, 'criterion': <class 'torch.nn.modules.loss.L1Loss'>, 'starter': 512, 'activation': RReLU(lower=0.125, upper=0.3333333333333333), 'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 0.0001}


,mean_loss,accuracy,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,neg_mean_loss,experiment_id,date,...,timesteps_since_restore,iterations_since_restore,experiment_tag,config.epochs,config.batch_size,config.criterion,config.starter,config.activation,config.optimizer,config.lr
trial_id,,,,,,,,,,,,,,,,,,,,,
b3bfb_00000,26154.810547,0.087,0.532060,True,None,None,25,-26154.810547,3931e1a377ab4b57ac65c5a274772c15,2021-07-07_13-18-38,...,0,25,"0_activation=ReLU6(),batch_size=6222,criterion...",25,6222,<class 'torch.nn.modules.loss.MSELoss'>,64,ReLU6(),<class 'torch.optim.adagrad.Adagrad'>,0.0010
b3bfb_00001,72.096039,0.002,0.549008,True,None,None,25,-72.096039,884403f1fd6b4abd81b4c82de10987ec,2021-07-07_13-18-56,...,0,25,"1_activation=RReLU(lower=0.125, upper=0.333333...",25,6222,<class 'torch.nn.modules.loss.L1Loss'>,512,"RReLU(lower=0.125, upper=0.3333333333333333)",<class 'torch.optim.sgd.SGD'>,0.0001


In [ ]:
# config = analysis.get_best_config(metric="mean_loss", mode="min")
# config['epochs'] = 100
# train(config,False,f'final-{config["epochs"]}')

In [105]:
# config = analysis.get_best_config(metric="accuracy", mode="min")
# config['epochs'] = 100
# model = train(config,False,f'final-{config["epochs"]}-min')

In [106]:
config = analysis.get_best_config(metric="accuracy", mode="max")
config['epochs'] = 100
model = train(config,False,f'final-{config["epochs"]}-max')

loss,71.97433
_runtime,66
_timestamp,1625644408
_step,199
val_accuracy,0.0


loss,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,▁▁▁▁▁▁██▆▃▂▅▁▂▁▂▂▁▁▂▁▂▁▂▁▂▂▁▂▂▁▂▁▅▁▁▁▂▁▁


100%|██████████| 100/100 [00:55<00:00,  1.81it/s]


In [107]:
test_df = pd.read_csv('./test.csv')

In [108]:
dates = []
times = []
for date_time in test_df['date_time']:
    date,time = date_time.split(' ')
    time = time.replace(':','')
    date = date.replace('-','')
    dates.append(date)
    times.append(time)

In [109]:
test_df['date'] = dates
test_df['time'] = times
test_df['date'] = test_df['date'].astype(int)
test_df['time'] = test_df['time'].astype(int)

In [110]:
date_time = test_df['date_time']

In [111]:
del test_df['date_time']

In [112]:
test_df = np.array(test_df)

In [113]:
preds = model(torch.tensor(test_df).to(device).float())

In [114]:
y_train

,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
7004,1.5,5.2,171.2
3634,0.5,2.0,23.5
367,1.4,4.6,81.4
4039,0.9,4.8,36.1
1803,0.7,5.0,67.9
...,...,...,...
4876,1.8,11.6,263.9
2225,2.0,11.7,137.2
339,2.6,8.9,165.6
2480,0.4,2.0,23.3


In [115]:
target_carbon_monoxides = []
target_benzenes = []
target_nitrogen_oxides = []
for pred in preds:
    target_carbon_monoxides.append(float(pred[0].cpu()))
    target_benzenes.append(float(pred[1].cpu()))
    target_nitrogen_oxides.append(float(pred[2].cpu()))

In [116]:
submission = pd.DataFrame({'date_time':date_time,'target_carbon_monoxide':target_carbon_monoxides,'target_benzene':target_benzenes,'target_nitrogen_oxides':target_nitrogen_oxides})

In [117]:
submission.to_csv('./sub-0.csv',index=False)